# Imports and data uploads

As is customary, let us first call the Python libraries needed here, and upload the needed data and code.

In [13]:
from model import setup, balance_calcs
from visuals import storage_plots
# import datetime
# import pandas as pd
# import numpy as np

# Preparing the set up.

In this tutorial we will build on the model we developed in Tutorial 1. Rather than share all the code in the Notebook again, this time we'll use auxiliary files to do the heavy lifting.

In [4]:
# Preparing the model
reservoir_name = 'Conowingo'
downstream_demand_names = ['Environmental']
direct_demand_names = ['Baltimore', 'Chester', 'Nuclear plant']

# Loading the model!
conowingo = setup.define_reservoir(reservoir_name, downstream_demand_names, direct_demand_names)

# Checking on the structure, e.g.:
print('Demand downstream of the dam is ' + conowingo.demand_downstream[0].name + '.')
print('Dead storage is ' + "{:.1f}".format(conowingo.dead_storage / 100**3) + ' hm3')

Demand downstream of the dam is Environmental.
Dead storage is 171.0 hm3


In [5]:
# Read flow and demand data
water_balance = setup.extract_flows(reservoir=conowingo)
print(water_balance)

            Total inflows (m3/s)  Baltimore demand (m3/s)  \
Date                                                        
1932-01-01            557.049006                13.139017   
1932-01-02            638.488257                13.139017   
1932-01-03            758.806538                13.139017   
1932-01-04            824.048553                13.139017   
1932-01-05            780.383975                13.139017   
...                          ...                      ...   
2001-12-27            775.966547                13.139017   
2001-12-28            716.529486                13.139017   
2001-12-29            668.362530                13.139017   
2001-12-30            580.920108                13.139017   
2001-12-31            481.867778                13.139017   

            Chester demand (m3/s)  Nuclear plant demand (m3/s)  \
Date                                                             
1932-01-01               1.472476                     0.622971   
1932-01-

In [11]:
# Computing the water balance for our standard operating policy (SOP)
balance_calcs.sop_full(reservoir=conowingo, water_flows=water_balance)
print(water_balance.columns)
print("{:.2f}".format(water_balance['Withdrawals Baltimore (m3/s)'].max()))

Index(['Total inflows (m3/s)', 'Baltimore demand (m3/s)',
       'Chester demand (m3/s)', 'Nuclear plant demand (m3/s)',
       'Environmental demand (m3/s)', 'Withdrawals Baltimore (m3/s)',
       'Withdrawals Chester (m3/s)', 'Withdrawals Nuclear plant (m3/s)',
       'Outflows (m3/s)', 'Storage (m3)'],
      dtype='object')
13.14


## Having tools to plot results

Now that we have our water balance, we can have the same visuals as before.

In [8]:
# Storage over the whole period
fig = storage_plots.timeseries(conowingo, water_balance)

NameError: name 'visuals' is not defined

In [ ]:
# Storage in the dry period
fig = visuals.plot_storage(conowingo, water_balance, first_date=datetime.date(1962, 1, 1), last_date=datetime.date(1968, 1, 1))

In [ ]:
# Outflows
fig = visuals.plot_flux(water_balance, 'Outflows', first_date=datetime.date(1962, 1, 1), last_date=datetime.date(1968, 1, 1))

In [ ]:
# And more, e.g., nuclear plant withdrawals
fig = visuals.plot_flux(water_balance, 'Withdrawals Nuclear plant', first_date=datetime.date(1960, 1, 1), last_date=datetime.date(1970, 1, 1))

In [ ]:
# A method to compute daily hydropower production has been added to the Reservoir class. Let's try it out!
daily_hydropower = conowingo.daily_production(water_balance)
# As in tutorial one we can use that to compute average annual production
print('Annual average hydropower production at Conowingo is ' + "{:.0f}".format(daily_hydropower.sum() / 70 / 1000) + ' GWh.')

In [ ]:
# We can also plot annual hydropower production, like this.
fig = visuals.plot_annual_hydropower(conowingo, water_balance)

# Performance indicators

Management objectives are as follows:
1) Produce hydropower
2) Meet environmental flows
3) Meet domestic and industrial demands
4) Avoid excessive flooding that would require evacuating the downstream town of “Port Deposit” (15,000 m3/s)
5) Maintain a water level compatible with recreation (hydraulic head over 106.5 ft, where 1ft = 0.3048 m) in June, July and August.

We explored objective (1) in Tutorial 1, and we will now focus on the other objectives.

For these objectives, we will compare the water flows / levels versus a threshold, and use the R-R-V indicators defined in the lecture. See the function below.


In [ ]:
# Function for performance indicators vs. dynamic threshold.

def rrv_indicators(time_series, dynamic_threshold, above_desirable, name, **kwargs):
    """
    Compute the RRV indicators for a time series vs. a threshold. Arguments:
        time_series: numpy vector
        dynamic_threshold: numpy vectors of same length as `time_series`
        above_desirable: boolean. If True we value staying at or above a threshold.
        name: String, the name of the site
        optional argument `vul_unit`: String, default as a percentage, to specify how vulnerability is evaluated
    Returns a pandas DataFrame with several performance metrics.
    """

    # Optional argument
    vul_unit = kwargs.pop("vul_unit", '%')
    print(vul_unit)

    # Local variables
    n_steps = len(time_series)
    tolerance = 1E-6  # for rounding errors

    # If above_desirable is false we need to change sign of all data now, so we compare a and b
    a = (2 * above_desirable - 1) * time_series
    b = (2 * above_desirable - 1) * dynamic_threshold
    b = b - tolerance

    # Initialise output
    indicators = pd.DataFrame(columns=['Name', 'Reliability (0-1)', 'Resilience (-)', 'Vulnerability', 'Failure count'])
    indicators.loc[0, 'Name'] = name

    # Reliability
    indicators.loc[0, 'Reliability (0-1)'] = 1 - np.sum(a < b) / n_steps

    # We need to count failure events to compute resilience and vulnerability
    event_count = 0
    # We also need to have the maximal amplitude or magnitude of failure
    magnitude = []
    # We use a while loop to count events and their magnitude
    t = 0
    while t < n_steps:

        if a[t] < b[t]:
            # New event! we need to update the count of failure events
            event_count = event_count + 1
            # We also need to keep track of the maximum amplitude of failure
            # By default failure is expressed in relative terms
            if vul_unit == '%':
                magnitude.append((b[t] - a[t]) / abs(b[t]))
            else:
                magnitude.append(b[t] - a[t])
            # Now while event lasts
            while a[t] < b[t]:
                t = t+1
                if t == n_steps:
                    break
                if vul_unit == '%':
                    magnitude[-1] = max(magnitude[-1], (b[t] - a[t]) / abs(b[t]))
                else:
                    magnitude[-1] = max(magnitude[-1], b[t] - a[t])

        # Time increment so while loop concludes
        t = t+1

    # Resilience
    indicators.loc[0, 'Resilience (-)'] = event_count / (n_steps * (1 - indicators.loc[0, 'Reliability (0-1)']))

    # Vulnerability (as a percentage)
    if vul_unit == '%':
        indicators.loc[0, 'Vulnerability'] = "{:.0f}".format(np.mean(magnitude) * 100) + '%'
    else:
        indicators.loc[0, 'Vulnerability'] = "{:.2f}".format(np.mean(magnitude)) + vul_unit

    # Finally, exporting the failure count
    indicators.loc[0, 'Failure count'] = event_count

    return indicators
 

In [ ]:
# Application to demands
metrics = pd.concat([rrv_indicators(water_balance['Withdrawals Baltimore (m3/s)'].to_numpy(), 
                                    water_balance['Baltimore demand (m3/s)'].to_numpy(), True, 'Baltimore'),
                     rrv_indicators(water_balance['Withdrawals Chester (m3/s)'].to_numpy(), 
                                    water_balance['Chester demand (m3/s)'].to_numpy(), True, 'Chester'),
                     rrv_indicators(water_balance['Withdrawals Nuclear plant (m3/s)'].to_numpy(), 
                                    water_balance['Nuclear plant demand (m3/s)'].to_numpy(), True, 'Nuclear'),
                     rrv_indicators(water_balance['Outflows (m3/s)'].to_numpy(), 
                                    water_balance['Environmental demand (m3/s)'].to_numpy(), True, 'Env. flows')],
                     axis=0, ignore_index=True)

print('Performance metrics for demands are:\n')
print(metrics)
print('\n')

In [ ]:
# Same for flooding
flooding_metrics = rrv_indicators(water_balance['Outflows (m3/s)'].to_numpy(), 15000*np.ones(len(water_balance)), False, 'Flooding')

metrics = pd.concat([metrics, flooding_metrics], axis=0, ignore_index=True)

print('Performance metrics including demands and flooding are:\n')
print(metrics)
print('\n')

#### Question 1
Comment on the reliability, resilience and vulnerability for the flooding objective. In particular, can we use different operating policies to avoid flooding the town downstream of Conowingo (Port Deposit)?

In [ ]:
# Summer recreation (lake levels need to stay above a certain level in June, July and August)

# We need time series of level objectives. We initialise at 0 requirement.
level_objective = pd.Series(index=water_balance.index, data=np.zeros(len(water_balance)))

# We set a level during summer months, to be compared with lake level (which coincide with hydraulic head)
summer_requirement = 106.5*0.3048
for month in np.arange(6, 9, 1):
    level_objective[level_objective.index.month == month] = summer_requirement

# Get hydraulic head time series, assuming linear relationship between depth and lake area
hydraulic_head = np.zeros(len(water_balance))
for t in range(len(water_balance)):
    depth = conowingo.get_depth(water_balance.iloc[t, -1])
    hydraulic_head[t] = conowingo.hydropower_plant.nominal_head - conowingo.total_lake_depth + depth

# Get the indicators
recreation_metrics = rrv_indicators(hydraulic_head, level_objective.to_numpy(), True, 'Recreation', vul_unit='m')

# We need to account for the fact that this requirement is for three months only, which impacts reliability
# Failure happens more often if measured in the shorter time window
recreation_metrics.iloc[0, 1] = 1 - (1-recreation_metrics.iloc[0, 1]) * len(level_objective) / (70*(30+31+31))

metrics = pd.concat([metrics, recreation_metrics], axis=0, ignore_index=True)
print('Performance metrics including demands, flooding and recreation are:\n')
print(metrics)
print('\n')

In [ ]:
# Add a new column, volumetric reliability
metrics.insert(5, 'Volumetric reliability', [0, 0, 0, 0, 'N/A', 'N/A'])
print(metrics)

In [ ]:
# Volumetric reliability is only defined for the demands, and it relies on the grand total supply / demand
totals = water_balance.sum(axis=0)

metrics.loc[0, 'Volumetric reliability'] = totals['Withdrawals Baltimore (m3/s)'] / totals['Baltimore demand (m3/s)']
metrics.loc[1, 'Volumetric reliability'] = totals['Withdrawals Chester (m3/s)'] / totals['Chester demand (m3/s)']
metrics.loc[2, 'Volumetric reliability'] = totals['Withdrawals Nuclear plant (m3/s)'] / totals['Nuclear plant demand (m3/s)']
metrics.loc[3, 'Volumetric reliability'] = np.sum(np.minimum(water_balance['Environmental demand (m3/s)'], water_balance['Outflows (m3/s)'])) / totals['Environmental demand (m3/s)']

print(metrics)


#### Question 2
Which objectives do you feel the chosen operating policy favours? Why?

# Prioritising current water uses with economic principles?

Until now we assumed that users took the water if it was available. But now we are the regulator and we want to make sure storage is regulated to make sure the most valuable uses are rewarded, even if that means some users have to scale back on water withdrawals even as they could take more. 

Before negotiating with the different users we need to have an idea which uses would be prioritised in practice. We need to know, depending on head (h) in the reservoir, how a quantity of water Q that we decide to allocate for releases / withdrawals breaks down between users. All prices in USD. Our starting assumptions are:

=> We prioritise ecological conservation and give water for meeting environmental flows a value of USD 10/m3.

=> Baltimore and Chester both have alternative sources of water supply so we assume a water value for their uses that is linear at USD 3/m3. If there is a water shortage for them, we assume it will be equal in proportion for both cities.

=> Average benefits from hydropower USD 60/MWh.

=> Average benefits from nuclear power production USD 40/MWh.

Hydropower is more valuable on average than nuclear power (per MWh), because turbines can be turned on or off almost instantly so hydropower can easily meet peak demand (more expensive) whereas nuclear power meets the base load.

#### Question 3
Given the average annual power production from Peach Bottom is around 20,000 GWh, and that all of it needs cooling water, what’s the average value of a m3 in USD?

In [ ]:
# If you have calculations you can use this to do them before looking up the answers.

#### Question 4
How much water does it take to produce 1MWh of hydropower? What does it mean for the value of water for hydropower?

In [ ]:
# If you have calculations you can use this to do them before looking up the answers.

#### Question 5 
With this in mind, how can we prioritise uses if the lake is full on a given day? If the water level is 3 metres lower than full?

#### Question 6
How sensitive is this order to the assumptions made? Were all those assumptions driven by economics alone?

#### Question 7
Does this evaluation support releasing water for hydropower production when the reservoir is not full?

#### Question 8
If we had not valued ecological flows, what would be the consequences?

#### Question 9
Overall, do the failure indicators reflect how valuable respective uses are? If not, what could be done to improve that?